### Sentiment analysis of movie (IMDB) reviews using dataset provided by the ACL 2011 paper, see http://ai.stanford.edu/~amaas/data/sentiment/.

#### Dataset can be downloaded separately from http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz, but wont be necessary as the download process has been embedded in the notebook and source file.

In [1]:
# !pip install nltk
# !pip install --upgrade gensim

import numpy as np
import os
import os.path

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
#nltk.download('punkt')>>> import nltk


import glob
from gensim.models import Word2Vec

import time

C:\Users\Rodolfo\Anaconda3\envs\flasky\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [63]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
print('On the MacOSX, you will need to install wget, see https://www.mkyong.com/mac/wget-on-mac-os-x/')

if not os.path.isfile('aclImdb_v1.tar.gz'):
  !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

if not os.path.isfile('aclImdb'):  
  !tar -xf aclImdb_v1.tar.gz 


On the MacOSX, you will need to install wget, see https://www.mkyong.com/mac/wget-on-mac-os-x/


In [2]:
time_beginning_of_notebook = time.time()
SAMPLE_SIZE=600
positive_sample_file_list = glob.glob(os.path.join('aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list = glob.glob(os.path.join('aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


# New Section

In [3]:
positive_strings = [load_doc(x) for x in positive_sample_file_list]
#print('\n Positive reviews \n ',positive_strings[:5])

negative_strings = [load_doc(x) for x in negative_sample_file_list]
#print('\n Negative reviews \n ', negative_strings[:5])
    

In [4]:
positive_labels = np.array(SAMPLE_SIZE * [1])


In [5]:
negative_labels = np.array(SAMPLE_SIZE * [0])


In [6]:
positive_tokenized = [word_tokenize(s) for s in positive_strings]
#print('\n Positive tokenized 1 \n {} \n\n Positive tokenized 2 \n {}'. format(positive_tokenized[1], positive_tokenized[2]))


In [7]:
negative_tokenized = [word_tokenize(s) for s in negative_strings]
#print('\n Negative tokenized 1 \n {} \n\n  Negative tokenized 2 \n {}'. format(negative_tokenized[1], negative_tokenized[2]))

In [8]:
# load doc into memory
with open('aclImdb/imdb.vocab', encoding='utf8') as f:
    #content = f.readlines()
    universe_vocabulary = [x.strip() for x in f.readlines()]

print("Word count across all reviews (before stripping tokens):", sum([len(token) for token in positive_tokenized]))

#Checking the not alphanumeric characters in vocabulary
non_alphanumeric_set = set()
for word in universe_vocabulary:
    non_alphanumeric_set |= set(re.findall('\W', word))
print('Non alphanumeric characters found in universe vocabulary', non_alphanumeric_set)


stripped_positive_tokenized = []
for tokens in positive_tokenized:
  stripped_positive_tokenized.append([token.lower() for token in tokens if token.lower() in universe_vocabulary])

print("Word count across all reviews (after stripping tokens):", sum([len(token) for token in stripped_positive_tokenized]))

Word count across all reviews (before stripping tokens): 161165
Non alphanumeric characters found in universe vocabulary {'?', ')', '}', ']', ';', '!', '(', '-', '[', "'", ':', '='}
Word count across all reviews (after stripping tokens): 139772


In [17]:
#print(positive_tokenized[0:5])
#print(stripped_positive_tokenized[0:5])

In [9]:
print("Word count across all reviews (before stripping tokens):", sum([len(token) for token in positive_tokenized]))
stripped_negative_tokenized = []
for tokens in negative_tokenized:
  stripped_negative_tokenized.append([token.lower() for token in tokens if token.lower() in universe_vocabulary])

print("Word count across all reviews (after stripping tokens):", sum([len(token) for token in stripped_negative_tokenized]))

Word count across all reviews (before stripping tokens): 161165
Word count across all reviews (after stripping tokens): 135014


In [19]:
#print(negative_tokenized[0:5])
#print(stripped_negative_tokenized[0:5])

In [20]:
#### Commenting out this bit as it is adding to the time to load the notebook, we can uncomment it when we need to reuse it again

# model_ted = Word2Vec(sentences=positive_tokenized, size=100, window=5, min_count=5, workers=1, sg=0, seed=42)
# model_ted.wv.most_similar("brother")

# print(np.linalg.norm(model_ted.wv['man'] - model_ted.wv['woman']))
# print(np.linalg.norm(model_ted.wv['father'] - model_ted.wv['mother']))
# print(np.linalg.norm(model_ted.wv['brother'] - model_ted.wv['sister']))
# print(np.linalg.norm(model_ted.wv['house'] - model_ted.wv['road']))  ### boat or ship does not exist in the corpus so we get an error if we use them

# print(np.linalg.norm(model_ted.wv['father'] - model_ted.wv['mother']))
# print(np.linalg.norm(model_ted.wv['sister'] - model_ted.wv['mother']))

In [10]:
features = np.array(stripped_positive_tokenized + stripped_negative_tokenized)
labels = np.concatenate([positive_labels, negative_labels])
# print(features.shape)
# print(features)
# print(labels.shape)
# print(labels)

from keras.preprocessing import text


# GitHub reference: https://github.com/tensorflow/workshops/blob/master/extras/keras-bag-of-words/keras-bow-model.ipynb
# Blog: https://cloud.google.com/blog/products/gcp/intro-to-text-classification-with-keras-automatically-tagging-stack-overflow-posts

vocab_size = 1000
tokenize = text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(features)
tokenized_features = tokenize.texts_to_matrix(features)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tokenized_features, labels, test_size=0.25)

print(x_train[1])
# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(y_test.shape)

C:\Users\Rodolfo\Anaconda3\envs\flasky\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

We have decided to do the use the below models and vectorisation techniques to test our their accuracy / score, the idea is to use a one model and one vectorization technique and plot a score.

**Simple models**

- Logistic Regression
- Random Forst
- LSTM
- GRU
- CNN

**Vectorisation techniques**
- Bag of Words
- Word2Vec
- TFIDF (probability scores)
- FastText
- Glove

## Logistic Regress model using Bag of Words vectorisation technique

In [11]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()

logisticRegr.fit(x_train, y_train)

score = logisticRegr.score(x_test, y_test)
print("Score: ", score)
y_test = logisticRegr.predict(x_test)
time_end_of_notebook = time.time()

Score:  0.82


## Logistic Regress model using TFIDF vectorisation technique.
## Introducing Pipeline: http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

## Introducing TfdfVectorizer: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

## Introducing  grid search hyperparameter optimization technique: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

<br>


In [13]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

df_positives = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(SAMPLE_SIZE)})
df_negatives = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(SAMPLE_SIZE)})

df = pd.concat([df_positives, df_negatives], ignore_index=True)

df = shuffle(df)

X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

stop = stopwords.words('english')
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

param_grid_1 = {'vect__ngram_range': [(1,1)], 'vect__stop_words': [None], 'vect__tokenizer': [tokenizer_porter],
            'clf__penalty': ['l1'], 'clf__C': [1.0]}
param_grid_2 = {'vect__ngram_range': [(1,1)],'vect__stop_words': [stop, None], 'vect__tokenizer': [word_tokenize, tokenizer_porter],
             'vect__use_idf':[False], 'vect__norm':[None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}
param_grid = [param_grid_1]

lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])

# I COULD NOT RUN THE FOLLOWING COUPLE OF INSTRUCTIONS .. IT TOOK TOO LONG IN MY LAPTOP
# gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=2, verbose=1, n_jobs=-1)
# gs_lr_tfidf.fit(X_train, y_train)

lr_tfidf.fit(X_train, y_train)

print('Test accuracy {}'.format(lr_tfidf.score(X_test, y_test)))
 

Test accuracy 0.8733333333333333


In [14]:

table_models_vectorization = pd.DataFrame(
     {'Models':                   ["Logistic Regression", "Logistic Regression", "Logistic Regression"], 
      'Vectorisation techniques': ["Bag of Words",        "Word2Vec", "TFIDF"], 
      'Score':                    [score,                 "Pending", lr_tfidf.score(X_test, y_test) ]},
    columns=['Models','Vectorisation techniques','Score']
)
print("Sample size:", SAMPLE_SIZE)

duration = time_end_of_notebook - time_beginning_of_notebook

print("Full notebook execution duration:", duration, "seconds")
print("Full notebook execution duration:", duration / 60, "minutes")

table_models_vectorization

Sample size: 600
Full notebook execution duration: 329.4903862476349 seconds
Full notebook execution duration: 5.491506437460582 minutes


,Models,Vectorisation techniques,Score
0,Logistic Regression,Bag of Words,0.82
1,Logistic Regression,Word2Vec,Pending
2,Logistic Regression,TFIDF,0.873333
